In [1]:
import subprocess
import os
import regex as re
import numpy as np
import itertools
import operator
from collections import Counter
from scipy import stats
from statistics import mean
import gzip
import pickle
import pysam
import mygene
pysam.__version__

'0.15.2'

In [3]:
bamfile = pysam.AlignmentFile("final.bam","rb")
split = pysam.AlignmentFile("SA_tag.bam", "wb", template=bamfile)

for line in bamfile:
    if line.has_tag("SA"):
        split.write(line)
    else:
        pass
    
bamfile.close()
split.close()

In [16]:

bamfile = pysam.AlignmentFile("SA_tag.bam")

human_human = 0
human_mouse = 0
mouse_mouse = 0
mouse_human = 0

for line in bamfile:

    a = line.get_tag("SA").split(";")[0]
    chrom, start, strand, cigar, mapq, nm = a.split(",")
    
    match = re.findall(r'ENST[0]', chrom)
    
    for m in match:
        if line.reference_name[:4] == "ENST":
            human_human += 1
        elif line.reference_name[:4] == "ENSM":
            human_mouse += 1
    
    match = re.findall(r'ENSMUST[0]', chrom)

    for m in match:
        if line.reference_name[:4] == "ENST":
            mouse_human += 1
        elif line.reference_name[:4] == "ENSM":
            mouse_mouse += 1

            
            

print("human_human:" + str(human_human))
print("human_mouse:" + str(human_mouse))
print("mouse_human:" + str(mouse_human))
print("mouse_mouse:" + str(mouse_mouse))

bamfile.close()

huamn_human:121111
huamn_mouse:22360
mouse_human:22738
mouse_mouse:133619


Now need  to seperate mouse and humand and do counting of the mouse human reads, are they completely random or are they at certain loci

In [34]:
# parse file and identify translocations - make bed file

bamfile = pysam.AlignmentFile("SA_tag.bam")

trans_list = []

n = 0
for line in bamfile:
    if n < 5:
        umi = line.query_name.split("_")[2]
        barcode = line.query_name.split("_")[1]

        chrom1 = line.reference_name
        start1 = line.reference_start
        end1 = line.reference_end
        sa = line.get_tag("SA").split(",")[0]

        chrom = chrom1 + "_" + sa + "_" + barcode
        
        trans_list.append(chrom)
        
print(trans_dict)
bamfile.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [41]:
out_table = open("translocations.txt", "w")
import collections 
trans_counter =collections.Counter(trans_list)
for k in trans_counter:
    ref, chimera, barcode = k.split("_")
    
    counter = trans_counter[k]
    
    out_table.write("%s\t%s\t%s\t%s\n"%(ref, chimera,  barcode, str(counter)))
